# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [16]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [17]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [18]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [19]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [20]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [21]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [22]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [23]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [24]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [25]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [26]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [27]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [28]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

2024-08-21 18:10:34,707 - 140032363940736 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [29]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive report for our sales team.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 


Search results: Title: Deeplearning.ai - Crunchbase Company Profile & Funding
Link: https://www.crunchbase.com/organization/deeplearning-ai
Snippet: Details. Deeplearning.ai is company tha

Thought: 
I need to gather more insights into DeepLearningAI's recent business developments and potential needs aligned with our offerings.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI recent business developments and potential needs"} 


Search results: Title: Deep learning recent news | AI Business
Link: https://aibusiness.com/ml/deep-learning
Snippet: Discover the latest AI startups, new and upcoming development in language modelling, and useful case studies and analysis.
---
Title: Deep Learning and Neural Networks Drive a Potential $7.9 Trillion ...
Link: https://www.prnewswire.com/news-releases/deep-learning-and-neural-networks-drive-a-potential-7-9-trillion-ai-economy-302130116.html
Snippet: One major aspect of the AI revolution is the rapid progression of deep machine learning and neural networks, which are each making major strides ...
---
Title: Advancements in Artificial Intelligence and Machine Learning
Link: https://online-engineering.case.ed

Final Answer:

Subject: Elevating AI Education with CrewAI: A Message from a Fellow Innovator

Dear Andrew Ng,

I hope this message finds you well. As a leader in the AI industry, your dedication to democratizing deep learning and advancing AI education has been truly inspiring. Your recent launch of new online courses at DeepLearningAI encapsulates the essence of modern AI learning: dynamic, application-focused, and boundary-pushing.

I wanted to reach out to introduce you to CrewAI, a cutting-edge AI solutions provider that shares your passion for making world-class AI education accessible globally. Our tools are designed to support organizations like DeepLearningAI in AI education, technology development, and application deployment.

At CrewAI, we believe in the power of personalized engagement strategies to drive growth and add value to our partners' operations. By leveraging our tools and strategic mindset, we can tailor solutions that align with DeepLearningAI's goals and further

- Display the final result as Markdown.

In [30]:
from IPython.display import Markdown
Markdown(result)

Subject: Elevating AI Education with CrewAI: A Message from a Fellow Innovator

Dear Andrew Ng,

I hope this message finds you well. As a leader in the AI industry, your dedication to democratizing deep learning and advancing AI education has been truly inspiring. Your recent launch of new online courses at DeepLearningAI encapsulates the essence of modern AI learning: dynamic, application-focused, and boundary-pushing.

I wanted to reach out to introduce you to CrewAI, a cutting-edge AI solutions provider that shares your passion for making world-class AI education accessible globally. Our tools are designed to support organizations like DeepLearningAI in AI education, technology development, and application deployment.

At CrewAI, we believe in the power of personalized engagement strategies to drive growth and add value to our partners' operations. By leveraging our tools and strategic mindset, we can tailor solutions that align with DeepLearningAI's goals and further enhance your impact on the AI community.

I would love the opportunity to discuss how CrewAI can support DeepLearningAI's mission and contribute to your continued success. Please let me know a convenient time for a call or meeting to explore potential collaboration opportunities.

Thank you for your time, Andrew. I look forward to the possibility of working together to elevate AI education and drive innovation in the industry.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

---

Note: This email draft is crafted to resonate with Andrew Ng's achievements in AI education and drive a connection between DeepLearningAI's recent initiatives and CrewAI's solutions. The tone is professional, engaging, and aligned with DeepLearningAI's corporate identity. The call to action prompts a specific response from Andrew Ng to discuss collaboration opportunities.